In [19]:
%load_ext autoreload
%autoreload 2

from init import init_code

init_code()

from app.services.quants.worldbrain import wqb_client, db
# 
# wqb_client.get_operator_list().json()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# d = {
#         "type": "REGULAR",
#         "settings": {
#             "instrumentType": "EQUITY",
#             "region": "USA",
#             "universe": "TOP3000",
#             "delay": 1,
#             "decay": 10,
#             "neutralization": "SUBINDUSTRY",
#             "truncation": 0.05,
#             "pasteurization": "ON",
#             "testPeriod": "P0Y0M",
#             "unitHandling": "VERIFY",
#             "nanHandling": "ON",
#             "maxTrade": "ON",
#             "language": "FASTEXPR",
#             "visualization": False,
#         },
#         "regular": """ranked_signal = ts_zscore(vec_sum(ern4_impliedee), 120);
#         signal = group_zscore(ranked_signal,SUBINDUSTRY);
#         ts_product(signal, 120)""",
#     }

# %load_ext autoreload
# %autoreload 2

# resp = await wqb_client.simulate_alpha(d)
# print(resp)




In [20]:

resp = await wqb_client.wqb.submit("j2vExPXj")
resp.content

b'<html>\r\n<head><title>400 The plain HTTP request was sent to HTTPS port</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<center>The plain HTTP request was sent to HTTPS port</center>\r\n</body>\r\n</html>\r\n'

Idea:
This factor measures the medium-term change in forecasted earnings strength relative to company size and compares it within each subindustry. It aims to capture firms showing improving earnings expectations adjusted for market capitalization.
Rationale for Data Used:
anl14_high_epsrep_fp1: Represents analysts’ high-end forward earnings-per-share (EPS) projections, indicating optimistic expectations about a firm’s future profitability.
cap: Denotes market capitalization, used here to scale earnings forecasts and control for size effects—preventing large-cap firms from dominating the signal.
Rationale for Operators Used:
ts_delta(): Calculates the change in the scaled EPS forecast over the past 120 periods, reflecting medium-term growth momentum or sentiment shifts.
group_rank(): Ranks companies within each subindustry to neutralize sectoral biases and focus on relative performance among comparable peers.


Idea: In normal market conditions, if a stock is shorted more, its likelihood of bouncing back may also increase (reversion). However, in extreme cases where the consensus in the market is high reflecting in extremely high/low level of short interest, it may potentially be better to follow that trend
Rationale for data used: shrt3_bar field is a vector data field representing the demand to borrow stock, with higher values indicating higher demand
Rationale for operators used:
vec_avg(): Calculates the average value of shrt3_bar for a given day
Conditional operator: Separates normal cases from extreme ones
ts_backfill: Handles NaN values in the data field, detected by checking the coverage with a visualization tool

In [18]:
d= {
  "complex": {
    "MATH_OP": ["", "log(x)", "signed_power(x, 2)", "sqrt(x)", "inverse(x)", "to_nan(x)"],
    "VECTOR_OP": ["vec_avg(x)"],
    "CROSS_OP": ["", "normalize(x)", "quantile(x)", "rank(x)", "scale(x)", "scale_down(x)", "zscore(x)"],
    "TS_OP": [
      "",
      "ts_decay_linear(x, {DAY})",
      "ts_rank(x, {DAY})",
      "ts_arg_max(x, {DAY})",
      "ts_arg_min(x, {DAY})",
      "ts_scale(x, {DAY})",
      "ts_zscore(x, {DAY})",
      "ts_av_diff(x, {DAY})",
      "ts_delta(x, {DAY})",
      "ts_quantile(x, {DAY})",
      "ts_std_dev(x, {DAY})",
      "last_diff_value(x, {DAY})",
      "days_from_last_change(x)"
    ],
    "GROUP_OP": [
      "",
      "group_rank(x, {GROUP_NAME})",
      "group_mean(x, 1, {GROUP_NAME})",
      "group_scale(x, {GROUP_NAME})",
      "group_zscore(x, {GROUP_NAME})",
      "group_max(x, {GROUP_NAME})",
      "group_min(x, {GROUP_NAME})"
    ],
    "GROUP_NAME": ["sector", "industry", "subindustry", "group_cartesian_product(country, subindustry)"],
    "DAY": ["5", "20", "60", "250"],
    "template": "{GROUP_OP}({CROSS_OP}({TS_OP}(winsorize(ts_backfill({MATH_OP}({FACTOR}), {DAY})))))"
  }
}
